In [13]:
import pandas as pd
import json
from pathlib import Path
import requests
import os

In [18]:
# Load the provinces data from the tat_thai_provinces.json file
with open('../../dataset/tat_provinces.json', 'r', encoding='utf-8') as f:
    provinces = json.load(f)

# API key for authentication
API_KEY = 'UmYQ8yiaTyArXEN5GMRJiCcvtu99ILDd'

# Base URL for fetching attractions list
BASE_URL = 'https://tatdataapi.io/api/v1/places'

LANGUAGE = 'en'

# Headers for the API request
headers = {
    'accept': 'application/json',
    'Accept-Language': LANGUAGE,
    'x-api-key': API_KEY,
}


In [20]:
# Loop through each province and fetch the data
for province in provinces:
    province_id = province['id']
    province_name = province['name']
    
    # Prepare the parameters for the API request
    params = {
        'provinceId': province_id,
        # 'place_category_id': 3,  # You can change this if necessary
        'pageSize': 100,
        # 'page': 1,
        # 'sort_by': 'placeId',
        'categoryCode': 'attractions'
    }
    
    # Send the GET request to fetch the attractions
    response = requests.get(BASE_URL, headers=headers, params=params)
    
    if response.status_code == 200:
        # Convert the response to JSON
        data = response.json()
        
        # Check if 'data' is available in the response
        if 'data' in data:
            attractions = data['data']
            
            # Transform the data into a DataFrame
            records = []
            for item in attractions:
                location = item.get("location", {})
                tags = item.get("tags", [])
                category = item.get("category", {})
                province = location.get("province", {})
                district = location.get("district", {})
                sub_district = location.get("subDistrict", {})
                
                record = {
                    "placeId": item.get("placeId"),
                    "place_name": item.get("name"),
                    "introduction": item.get("introduction"),
                    "category_name": category.get("name"),
                    "categoryId": category.get("categoryId"),
                    "latitude": item.get("latitude"),
                    "longitude": item.get("longitude"),
                    "postcode": location.get("postcode") if location else None,
                    "thumbnail_url": item.get("thumbnailUrl", [None])[0] if item.get("thumbnailUrl") else None,
                    "tags": ",".join(tags) if tags else "",  # Safely handle tags
                    "province_Id": province.get("provinceId") if province else None,
                    "province_name": province.get("name") if province else None,
                    "district_Id": district.get("districtId") if district else None,
                    "district_name": district.get("name") if district else None,
                    "sub_district_Id": sub_district.get("subDistrictId") if sub_district else None,
                    "sub_district_name": sub_district.get("name") if sub_district else None,
                    "updated_at": item.get("updatedAt"),
                }
                records.append(record)
            
            # Create a DataFrame
            df = pd.DataFrame(records)
            
            # Save the DataFrame to a CSV file
            csv_filename = f"data/{province_id}_attractions_{LANGUAGE}.csv"
            df.to_csv(csv_filename, index=False, encoding='utf-8')
            print(f"Data for {province_name} saved to {csv_filename}")
        else:
            print(f"No data found for {province_name} (ID: {province_id})")
    else:
        print(f"Failed to fetch data for {province_name} (ID: {province_id})")

Failed to fetch data for กระบี่ (ID: 344)
Failed to fetch data for กรุงเทพมหานคร (ID: 219)
Failed to fetch data for กาญจนบุรี (ID: 222)
Failed to fetch data for กาฬสินธุ์ (ID: 574)
Failed to fetch data for กำแพงเพชร (ID: 103)
Failed to fetch data for ขอนแก่น (ID: 575)
Failed to fetch data for จันทบุรี (ID: 463)
Failed to fetch data for ฉะเชิงเทรา (ID: 220)
Failed to fetch data for ชลบุรี (ID: 464)
Failed to fetch data for ชัยนาท (ID: 221)
Failed to fetch data for ชัยภูมิ (ID: 573)
Failed to fetch data for ชุมพร (ID: 343)
Failed to fetch data for ตรัง (ID: 355)
Failed to fetch data for ตราด (ID: 466)
Failed to fetch data for ตาก (ID: 115)
Failed to fetch data for นครนายก (ID: 224)
Failed to fetch data for นครปฐม (ID: 225)
Failed to fetch data for นครพนม (ID: 579)
Failed to fetch data for นครราชสีมา (ID: 580)
Failed to fetch data for นครศรีธรรมราช (ID: 345)
Failed to fetch data for นครสวรรค์ (ID: 107)
Failed to fetch data for นนทบุรี (ID: 226)
Failed to fetch data for นราธิวาส (ID: 346)


In [17]:
# Directory where the CSV files are stored (you can set this to the path where your CSV files are)
csv_directory = './data/'  # Assuming the files are in the current working directory

# Initialize an empty list to hold DataFrames
df_list = []

# Loop through all CSV files in the directory
for filename in os.listdir(csv_directory):
    if filename.endswith(f"_attractions_{LANGUAGE}.csv"):  # Only process the relevant CSV files
        file_path = os.path.join(csv_directory, filename)
        
        # Read each CSV file into a DataFrame and append it to the list
        df = pd.read_csv(file_path)
        df_list.append(df)

# Concatenate all DataFrames into one
merged_df = pd.concat(df_list, ignore_index=True)

# Save the merged DataFrame to a single CSV file
merged_df.to_csv(f"tat_attractions_{LANGUAGE}.csv", index=False, encoding='utf-8')

print(f"All files have been merged into 'tat_attractions_{LANGUAGE}.csv'")

EmptyDataError: No columns to parse from file

In [2]:
# Define the path to the JSON file
json_file = Path("../../dataset/tat_attractions.json")

# Check if the file exists
if not json_file.is_file():
    raise FileNotFoundError(f"The file '{json_file}' does not exist. Please provide the correct file path.")

# Load the JSON file
with json_file.open('r', encoding='utf-8') as file:
    data = json.load(file)

In [7]:

records = []
for item in data['data']:  # Added safeguard for 'data' key
    if not item:  # Ensure item is not None or empty
        continue
    
    location = item.get("location", {})  # Safe way to access location, defaults to empty dict if None
    tags = item.get("tags", [])  # Ensure tags defaults to an empty list
    category = item.get("category", {})  # Ensure category defaults to an empty dict
    province = location.get("province", {})  # Safely get province
    district = location.get("district", {})  # Safely get district
    sub_district = location.get("subDistrict", {})  # Safely get subDistrict
    
    record = {
        "placeId": item.get("placeId"),
        "place_name": item.get("name"),
        "introduction": item.get("introduction"),
        "category_name": category.get("name"),  # Safely access category name
        "categoryId": category.get("categoryId"),  # Safely access category ID
        "latitude": item.get("latitude"),
        "longitude": item.get("longitude"),
        "postcode": location.get("postcode") if location else None,
        "thumbnail_url": item.get("thumbnailUrl", [None])[0] if item.get("thumbnailUrl") else None,
        "tags": ",".join(tags) if tags else "",  # Safely handle tags and avoid errors when None
        "province_Id": province.get("provinceId") if province else None,  # Safely get province ID
        "province_name": province.get("name") if province else None,  # Safely get province name
        "district_Id": district.get("districtId") if district else None,  # Safely get district ID
        "district_name": district.get("name") if district else None,  # Safely get district name
        "sub_district_Id": sub_district.get("subDistrictId") if sub_district else None,  # Safely get sub-district ID
        "sub_district_name": sub_district.get("name") if sub_district else None,  # Safely get sub-district name
        "updated_at": item.get("updatedAt"),
    }
    records.append(record)

# Create a DataFrame from the extracted records
df = pd.DataFrame(records)

# Save the DataFrame to a CSV file (optional)
df.to_csv("tat_attractions.csv", index=False, encoding='utf-8')

# Display the DataFrame to verify
print(df.head())

  placeId                             place_name  \
0    1858  วัดสระเกศ ราชวรมหาวิหาร (วัดภูเขาทอง)   
1   33400               ศาลเจ้าพ่อ-เจ้าแม่ชุมแสง   
2   33586                   ตลาดเก่าร้อยปีชุมแสง   
3   33885                            ตลาดเจ้าค่ะ   
4   33325                 อุทยานเทวสถานพระพิฆเนศ   

                                        introduction      category_name  \
0  วัดสระเกศ หรือ วัดภูเขาทอง (Temple of the Gold...  สถานที่ท่องเที่ยว   
1                                               None  สถานที่ท่องเที่ยว   
2                                               None  สถานที่ท่องเที่ยว   
3  ตั้งอยู่ตำบลเขาดิน บริเวณริมแม่น้ำปิงด้านหน้าว...  สถานที่ท่องเที่ยว   
4                                               None  สถานที่ท่องเที่ยว   

   categoryId  latitude  longitude postcode  \
0           3  13.75389  100.50841    10100   
1           3  15.89933  100.30886    60120   
2           3  15.89541  100.30847    60120   
3           3  15.81700  100.07715    60230   
4

In [8]:
df

,placeId,place_name,introduction,category_name,categoryId,latitude,longitude,postcode,thumbnail_url,tags,province_Id,province_name,district_Id,district_name,sub_district_Id,sub_district_name,updated_at
0,1858,วัดสระเกศ ราชวรมหาวิหาร (วัดภูเขาทอง),วัดสระเกศ หรือ วัดภูเขาทอง (Temple of the Gold...,สถานที่ท่องเที่ยว,3,13.75389,100.50841,10100,https://dmc.tatdataapi.io/assets/04e68492-da7d...,,219.0,กรุงเทพมหานคร,1008.0,ป้อมปราบศัตรูพ่าย,100804.0,บ้านบาตร,2024-12-20T09:19:14.997Z
1,33400,ศาลเจ้าพ่อ-เจ้าแม่ชุมแสง,None,สถานที่ท่องเที่ยว,3,15.89933,100.30886,60120,https://dmc.tatdataapi.io/assets/3b6743d6-dc9f...,ศาลเจ้าพ่อเจ้าแม่ชุมแสง,107.0,นครสวรรค์,6003.0,ชุมแสง,600301.0,ชุมแสง,2024-12-16T08:31:55.955Z
2,33586,ตลาดเก่าร้อยปีชุมแสง,None,สถานที่ท่องเที่ยว,3,15.89541,100.30847,60120,None,"ตลาดเก่าร้อยปีชุมแสง,นครสวรรค์",107.0,นครสวรรค์,6003.0,ชุมแสง,600301.0,ชุมแสง,2024-12-16T08:24:13.337Z
3,33885,ตลาดเจ้าค่ะ,ตั้งอยู่ตำบลเขาดิน บริเวณริมแม่น้ำปิงด้านหน้าว...,สถานที่ท่องเที่ยว,3,15.81700,100.07715,60230,None,ตลาดเจ้าค่ะ,107.0,นครสวรรค์,6006.0,เก้าเลี้ยว,600604.0,เขาดิน,2024-12-16T08:05:51.745Z
4,33325,อุทยานเทวสถานพระพิฆเนศ,None,สถานที่ท่องเที่ยว,3,15.64627,100.10917,60000,https://dmc.tatdataapi.io/assets/f8d3fde8-b638...,,107.0,นครสวรรค์,6001.0,เมืองนครสวรรค์,600102.0,กลางแดด,2024-12-16T07:49:49.078Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,31218,วิสาหกิจชุมชนท่องเที่ยวบ้านธรรมชาติล่างตำบลคลอ...,ชุมชนบ้านธรรมชาติล่าง ตั้งอยู่ในตำบลคลองใหญ่ ม...,สถานที่ท่องเที่ยว,3,12.19112,102.30377,23120,https://dmc.tatdataapi.io/assets/9900e29a-a195...,"ตราด,ตำบลคลองใหญ่ ,บ้านธรรมชาติล่าง,วิสาหกิจชุ...",466.0,ตราด,2305.0,แหลมงอบ,230507.0,คลองใหญ่,2024-07-15T05:27:18.658Z
196,29375,นิเวศพิพิธภัณฑ์บ้านช้างทูน,เป็นชุมชนที่นำเสนออัตลักษณ์ของชาติพันธุ์และวิถ...,สถานที่ท่องเที่ยว,3,12.61232,102.47859,23140,https://dmc.tatdataapi.io/assets/3a9da8ef-0403...,"ตราด,นิเวศพิพิธภัณฑ์บ้านช้างทูน,บ้านช้างทูน",466.0,ตราด,2304.0,บ่อไร่,230402.0,ช้างทูน,2024-07-15T05:27:17.968Z
197,31510,กลุ่มท่องเที่ยวเชิงอนุรักษ์บ้านพุมเรียง,None,สถานที่ท่องเที่ยว,3,9.38099,99.24567,84110,https://dmc.tatdataapi.io/assets/6818ae1d-0868...,"กลุ่มท่องเที่ยวเชิงอนุรักษ์บ้านพุมเรียง ,สุราษ...",354.0,สุราษฎร์ธานี,8406.0,ไชยา,840602.0,พุมเรียง,2024-07-15T05:27:16.542Z
198,33650,ถนนคนเดินตะกั่วป่า,ถนนคนเดินตะกั่วป่า จัดขึ้นทุกวันอาทิตย์ เวลา 1...,สถานที่ท่องเที่ยว,3,8.82992,98.36489,82110,https://dmc.tatdataapi.io/assets/a9ad0f54-3068...,,348.0,พังงา,8205.0,ตะกั่วป่า,820501.0,ตะกั่วป่า,2024-07-15T05:27:15.913Z
